In [1]:
print("ok")

ok


In [60]:
# !pip install -U langchain

In [44]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceHubEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PC
import pinecone
from pinecone import Pinecone, ServerlessSpec
from langchain.document_loaders import DirectoryLoader, PyPDFLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [16]:
PINECONE_API_KEY = "c3f5b6e3-d09b-4be8-95ee-3e44971b8472"
PINECONE_API_ENV = "us-east-1-aws"

In [5]:
#Extract data from pdf
def load_pdf(data):

    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [6]:
extracted_data = load_pdf("data/")

In [25]:
#extracted_data

In [7]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("length of my chunks:", len(text_chunks))

length of my chunks: 7020


In [98]:
#text_chunks

In [9]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [13]:
query_result = embeddings.embed_query("Hello world")
# query_result = embeddings.embed_
print("Length", len(query_result))
print("Response", query_result)

Length 384
Response [-0.03447723761200905, 0.031023213639855385, 0.006734990980476141, 0.02610895223915577, -0.03936200216412544, -0.16030248999595642, 0.06692393124103546, -0.006441502831876278, -0.04745049029588699, 0.014758865348994732, 0.07087529450654984, 0.05552753433585167, 0.019193345680832863, -0.026251327246427536, -0.010109513066709042, -0.026940496638417244, 0.022307435050606728, -0.022226642817258835, -0.1496925801038742, -0.01749304123222828, 0.007676258217543364, 0.05435232073068619, 0.0032544711139053106, 0.031725890934467316, -0.0846213549375534, -0.029405983164906502, 0.05159558728337288, 0.04812406003475189, -0.0033148040529340506, -0.05827920883893967, 0.04196924716234207, 0.022210638970136642, 0.1281888484954834, -0.022338991984725, -0.011656233109533787, 0.06292837113142014, -0.03287629410624504, -0.09122603386640549, -0.031175388023257256, 0.052699580788612366, 0.0470348484814167, -0.08420310914516449, -0.030056176707148552, -0.020744847133755684, 0.0095178894698

In [54]:
#from pinecone import Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY,
    serverless_spec=ServerlessSpec(
        cloud='aws',
        region="us-east-1"  # Set your desired region here
        ) 
)

 
index_name="medical-chatbot"
index = pc.Index(index_name)  
for i, t in zip(range(len(text_chunks)), text_chunks):
   query_result = embeddings.embed_query(t.page_content)
   index.upsert(
   vectors=[
        {
            "id": str(i),  # Convert i to a string
            "values": query_result, 
            "metadata": {"text":str(text_chunks[i].page_content)} # meta data as dic
        }
    ],
    namespace="real" 
)


index.describe_index_stats() 

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'real': {'vector_count': 7020}},
 'total_vector_count': 7020}

In [55]:
# Assuming your Pinecone index is already created and the embeddings model is ready
index = pc.Index(index_name)  # Reconnect to the existing index

# Define your query
query = "What are Allergies"

# Embed the query text using the embeddings model
query_embedding = embeddings.embed_query(query)

# Perform the similarity search in the Pinecone index
search_result = index.query(
    vector=query_embedding,
    top_k=3,  # The number of top similar documents to return
    include_metadata=True,  # To include metadata like the original text
    namespace="real"  # Use the namespace where your data is stored
)

# Print the search results
print("Results:", search_result)


Results: {'matches': [{'id': '1372',
              'metadata': {'text': 'GALE ENCYCLOPEDIA OF MEDICINE 2 '
                                   '117Allergies\n'
                                   'Allergic rhinitis is commonly triggered '
                                   'by\n'
                                   'exposure to household dust, animal fur,or '
                                   'pollen. The foreign substance thattriggers '
                                   'an allergic reaction is calledan '
                                   'allergen.\n'
                                   'The presence of an allergen causes the\n'
                                   "body's lymphocytes to begin producingIgE "
                                   'antibodies. The lymphocytes of an allergy '
                                   'sufferer produce an unusuallylarge amount '
                                   'of IgE.\n'
                                   'IgE molecules attach to mast\n'
        

In [56]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [57]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [58]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [59]:
# Reconnect to the existing Pinecone index
index = pc.Index(index_name)  # Reconnect to your Pinecone index

def retrieve_documents(query, top_k=2):
    # Embed the user's query
    query_embedding = embeddings.embed_query(query)
    
    # Perform similarity search in the Pinecone index
    search_result = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True,
        namespace="real"
    )
    
    # Extract the retrieved documents (metadata["text"])
    documents = [match['metadata']['text'] for match in search_result['matches']]
    return documents

def generate_response(llm, documents, query):
    # Combine the retrieved documents with the query to generate a response
    context = "\n\n".join(documents)
    
    # Generate the response using the LLM
    full_prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    response = llm(full_prompt)
    
    return response

while True:
    user_input = input("Input Prompt: ")
    
    # Retrieve relevant documents based on the user's input
    retrieved_docs = retrieve_documents(user_input, top_k=2)
    
    # Generate a response from the LLM using the retrieved documents
    final_response = generate_response(llm, retrieved_docs, user_input)
    
    # Print the response
    print("Response:", final_response)


Response:  Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria.Acne vulgaris, the medical term for common acne, is a chronic inflammatory skin disorder that affects primarily the face, particularly the areas around the mouth, nose, and cheeks. It is caused by a combination of factors, including hormonal changes, excess oil production, clogged pores, and bacterial infections.Acne can range from mild to severe and can cause physical and emotional distress. Treatment options include topical creams and gels, oral medications, and blue light therapy. In severe cases, corticosteroid injections or isotretinoin may be prescribed.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\meven\anaconda3\envs\mchatbot\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\meven\AppData\Local\Temp\ipykernel_5852\1028328379.py", line 38, in <module>
    final_response = generate_response(llm, retrieved_docs, user_input)
  File "C:\Users\meven\AppData\Local\Temp\ipykernel_5852\1028328379.py", line 26, in generate_response
    response = llm(full_prompt)
  File "c:\Users\meven\anaconda3\envs\mchatbot\lib\site-packages\langchain\llms\base.py", line 382, in __call__
  File "c:\Users\meven\anaconda3\envs\mchatbot\lib\site-packages\langchain\llms\base.py", line 234, in generate
  File "c:\Users\meven\anaconda3\envs\mchatbot\lib\site-packages\langchain\llms\base.py", line 178, in _generate_helper
  File "c:\Users\meven\anaconda3\envs\mchatbot\lib\site-packages\langchain\llms\base.py", line 165, in _generate_helper
  File "c:\Users\meven\